In [1]:
# !pip install scikit-learn==1.2.2 

In [2]:
import pandas as pd
import numpy as np
import sklearn
# # # Графика
# import matplotlib.pyplot as plt
# import seaborn as sns

# Деление выборки
from sklearn.model_selection import train_test_split

# from sklearn import preprocessing #предобработка данных
from sklearn.linear_model import LogisticRegression #линейные модели
from sklearn import tree #деревья решений
from sklearn.ensemble import RandomForestClassifier, StackingClassifier# онсамбли
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
print(sklearn.__version__) # 1.3.2
# import xgboost as xgb
# import catboost as cat

from sklearn.metrics import f1_score, make_scorer #метрики

1.2.2


(признак 'Family'). Семейство 'Dendrobatidae' будет классом 1, все остальные семейства — классом 0

In [3]:
Frogs_data = pd.read_csv('../../data/Frogs_MFCCs.zip')
Frogs_data.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [4]:
# Frogs_data.info()
# print(Frogs_data.isnull().sum().sum(),': суммарно пропусков в данных')

Разделите выборку на обучающую и тестовую в соотношении 80/20, параметр random_state = 31.

In [5]:
X=Frogs_data.drop(['Family','Genus','Species','RecordID'],axis=1)
y=Frogs_data['Family']

In [6]:
y = np.where((y == 'Dendrobatidae'), 0, 1)
# Frogs_data.insert(loc = Frogs_data.shape[1], column= 'y', value = temp)
# Frogs_data['y']=Frogs_data['Family'].apply(lambda x: 1 if x=="Dendrobatidae" else 0 )

In [11]:
# Формируем обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=31) # , stratify=y

### Задание 7.5
Для начала обучите на наших данных случайный лес с десятью решающими деревьями. Воспользуйтесь параметрами по умолчанию. В качестве значения random_state возьмите число 42.

Оцените значение $F_1$-меры и введите его в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.

In [14]:
#Создаем объект класса дерево решений
dtc = RandomForestClassifier( random_state=42)
#Обучаем дерево по алгоритму CARTrint
dtc.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = dtc.predict(X_train)
print('Train: {:.3f}'.format(f1_score(y_train, y_train_pred)))

y_test_pred = dtc.predict(X_test)
print('Test: {:.3f}'.format(f1_score(y_test, y_test_pred)))

Train: 1.000
Test: 0.999


In [13]:
model_rfc = RandomForestClassifier(10, random_state=42)
model_rfc.fit(X_train, y_train)
pred = model_rfc.predict(X_test)
f1_score(y_test, pred)

0.9985141158989599

### Задание 7.6
Теперь попробуем улучшить качество нашего предсказания за счёт использования стекинга.

В качестве базовых моделей выберите следующие:

случайный лес с десятью деревьями, random_state = 31;
KNN, количество соседей = 11;
наивный байесовский классификатор с параметрами по умолчанию, в качестве метода возьмите GaussianNB().
В качестве метамодели выберите логистическую регрессию.

Обучите модели и сделайте предсказание целевой метки для тестового набора данных.

Рассчитайте -меру для тестового набора данных и введите её в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.

In [10]:
estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', KNeighborsClassifier(n_neighbors= 11)),
              ('nb', GaussianNB() )               
]
metamodel = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
metamodel.fit(X_train, y_train)
pred = metamodel.predict(X_test)
f1_score(y_test, pred)

0.9988734509951183